## Kotak Bank Statement Table Extraction

In [15]:
import pandas as pd
import tabula
import PyPDF2
import re
from typing import List
import numpy as np

class BankStatementTableExtractor:
    def __init__(self, pdf_path: str):
        self.pdf_path = pdf_path
        self.tables = []
        
    def extract_tables_with_tabula(self) -> List[pd.DataFrame]:
        """
        Extract tables using tabula-py library
        """
        try:
            tables = tabula.read_pdf(
                self.pdf_path, 
                pages='all', 
                multiple_tables=True,
                pandas_options={'header': 0}
            )
            
            print(f"Found {len(tables)} tables using tabula")
            return tables
            
        except Exception as e:
            print(f"Error extracting tables with tabula: {e}")
            return []
    
    def extract_transaction_table_from_text(self, text: str) -> pd.DataFrame:
        """
        Extract transaction table from text using regex patterns
        """
        transaction_pattern = r'(\d{2}\s+\w{3},\s+\d{4})\s+(.*?)\s+([\w-]+)\s+([-+]?\d+\.?\d*)\s+([-+]?\d+\.?\d*)\s+(\d+\.?\d*)'
        
        matches = re.findall(transaction_pattern, text)
        
        if matches:
            df = pd.DataFrame(matches, columns=[
                'DATE', 'TRANSACTION_DETAILS', 'CHEQUE_REFERENCE', 
                'DEBIT', 'CREDIT', 'BALANCE'
            ])
            
            for col in ['DEBIT', 'CREDIT', 'BALANCE']:
                df[col] = pd.to_numeric(df[col].replace('', '0'), errors='coerce')
            
            return df
        
        return pd.DataFrame()
    
    def extract_tables_from_text(self) -> List[pd.DataFrame]:
        """
        Extract tables by parsing PDF text content
        """
        tables = []
        
        try:
            with open(self.pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                
                for page in pdf_reader.pages:
                    text = page.extract_text()
                    
                    if 'DATE TRANSACTION DETAILS' in text:
                        lines = text.split('\n')
                        table_data = []
                        in_table = False
                        
                        for line in lines:
                            if 'DATE TRANSACTION DETAILS' in line:
                                in_table = True
                                continue
                            elif 'SUMMARY' in line or 'Page' in line:
                                in_table = False
                                continue
                            
                            if in_table and line.strip():
                                parts = line.strip().split()
                                if len(parts) >= 6 and re.match(r'\d{2}\s+\w{3}', line):
                                    date_match = re.match(r'(\d{2}\s+\w{3},\s+\d{4})', line)
                                    if date_match:
                                        date = date_match.group(1)
                                        rest = line[date_match.end():].strip()
                                        
                                        balance_match = re.search(r'(\d+\.\d+)$', rest)
                                        if balance_match:
                                            balance = balance_match.group(1)
                                            
                                            amounts_section = rest[:balance_match.start()].strip()
                                            amounts = re.findall(r'[-+]?\d+\.\d+', amounts_section)
                                            
                                            debit = amounts[-2] if len(amounts) >= 2 and amounts[-2].startswith('-') else '0'
                                            credit = amounts[-1] if len(amounts) >= 1 and not amounts[-1].startswith('-') else '0'
                                            
                                            details_section = amounts_section
                                            for amt in amounts:
                                                details_section = details_section.replace(amt, '')
                                            
                                            reference_match = re.search(r'(UPI-\w+|NEFT\w+-\w+|\w+-\w+)$', details_section)
                                            reference = reference_match.group(1) if reference_match else ''
                                            
                                            transaction_details = details_section.replace(reference, '').strip()
                                            
                                            table_data.append([
                                                date, transaction_details, reference, 
                                                debit, credit, balance
                                            ])
                        
                        if table_data:
                            df = pd.DataFrame(table_data, columns=[
                                'DATE', 'TRANSACTION_DETAILS', 'CHEQUE_REFERENCE',
                                'DEBIT', 'CREDIT', 'BALANCE'
                            ])
                            
                            for col in ['DEBIT', 'CREDIT', 'BALANCE']:
                                df[col] = pd.to_numeric(df[col].replace('', '0'), errors='coerce')
                            
                            tables.append(df)
            
        except Exception as e:
            print(f"Error extracting from text: {e}")
        
        return tables
    
    def clean_and_standardize_tables(self, tables: List[pd.DataFrame]) -> List[pd.DataFrame]:
        """
        Clean and standardize table formats
        """
        cleaned_tables = []
        
        for i, table in enumerate(tables):
            if table.empty:
                continue
                
            table = table.dropna(how='all')
            
            if len(table) < 2:
                continue
            
            table.columns = [col.strip().upper().replace(' ', '_') for col in table.columns]
            
            cleaned_tables.append(table)
            print(f"Table {i+1}: {table.shape[0]} rows, {table.shape[1]} columns")
        
        return cleaned_tables
    
    def stack_tables_vertically(self, tables: List[pd.DataFrame]) -> pd.DataFrame:
        """
        Stack all tables vertically into a single DataFrame
        """
        if not tables:
            return pd.DataFrame()
        
        all_columns = set()
        for table in tables:
            all_columns.update(table.columns)
        
        standardized_tables = []
        for table in tables:
            for col in all_columns:
                if col not in table.columns:
                    table[col] = np.nan
            
            table = table.reindex(columns=sorted(all_columns))
            standardized_tables.append(table)
        
        combined_table = pd.concat(standardized_tables, ignore_index=True)
        
        print(f"Combined table: {combined_table.shape[0]} rows, {combined_table.shape[1]} columns")
        return combined_table
    
    def process_pdf(self) -> pd.DataFrame:
        """
        Main method to process the PDF and return stacked tables
        """
        print(f"Processing PDF: {self.pdf_path}")
        
        tables = self.extract_tables_with_tabula()
        
        if not tables or all(table.empty for table in tables):
            print("Tabula extraction failed, trying text extraction...")
            tables = self.extract_tables_from_text()
        
        cleaned_tables = self.clean_and_standardize_tables(tables)
        
        final_table = self.stack_tables_vertically(cleaned_tables)
        
        return final_table

def main():
    pdf_path = "bank-statements/kotak.pdf"
    
    try:
        extractor = BankStatementTableExtractor(pdf_path)
        combined_table = extractor.process_pdf()
        
        if not combined_table.empty:
            output_file = "extracted_tables_combined.csv"
            combined_table.to_csv(output_file, index=False)
            print(f"Combined table saved to: {output_file}")
            
            print("\nTable Info:")
            print(f"Shape: {combined_table.shape}")
            print(f"Columns: {list(combined_table.columns)}")
            print("\nFirst few rows:")
            print(combined_table.head())
            
        else:
            print("No tables were extracted from the PDF")
            
    except Exception as e:
        print(f"Error processing PDF: {e}")

if __name__ == "__main__":
    main()

def extract_with_camelot(pdf_path: str) -> pd.DataFrame:
    """
    Alternative method using camelot-py library
    """
    try:
        import camelot
        
        tables = camelot.read_pdf(pdf_path, pages='all')
        
        print(f"Found {len(tables)} tables with camelot")
        
        dfs = [table.df for table in tables]
        
        if dfs:
            combined = pd.concat(dfs, ignore_index=True)
            return combined
        else:
            return pd.DataFrame()
            
    except ImportError:
        print("Camelot not installed. Install with: pip install camelot-py[cv]")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error with camelot: {e}")
        return pd.DataFrame()

def process_bank_statement():
    """
    Specific function for processing the kotak bank statement
    """
    pdf_path = "bank-statements/kotak.pdf"
    
    print("For the Kotak bank statement, use the BankStatementTableExtractor class above")
    print("It will handle the specific format and extract transaction tables")

Processing PDF: bank-statements/kotak.pdf


Got stderr: Jun 12, 2025 10:49:24 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Jun 12, 2025 10:49:24 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode
Jun 12, 2025 10:49:25 AM org.apache.pdfbox.rendering.TTFGlyph2D getPathForGID
Jun 12, 2025 10:49:25 AM org.apache.pdfbox.rendering.Type1Glyph2D getPathForCharacterCode
Jun 12, 2025 10:49:26 AM org.apache.pdfbox.rendering.Type1Glyph2D getPathForCharacterCode
Jun 12, 2025 10:49:26 AM org.apache.pdfbox.rendering.Type1Glyph2D getPathForCharacterCode
Jun 12, 2025 10:49:27 AM org.apache.pdfbox.rendering.Type1Glyph2D getPathForCharacterCode
Jun 12, 2025 10:49:27 AM org.apache.pdfbox.rendering.Type1Glyph2D getPathForCharacterCode
Jun 12, 2025 10:49:27 AM org.apache.pdfbox.rendering.Type1Glyph2D getPathForCharacterCode



Found 4 tables using tabula
Table 1: 24 rows, 6 columns
Table 2: 40 rows, 6 columns
Table 3: 41 rows, 6 columns
Table 4: 13 rows, 6 columns
Combined table: 118 rows, 6 columns
Combined table saved to: extracted_tables_combined.csv

Table Info:
Shape: (118, 6)
Columns: ['BALANCE', 'CHEQUE/REFERENCE#', 'CREDIT', 'DATE', 'DEBIT', 'TRANSACTION_DETAILS']

First few rows:
    BALANCE CHEQUE/REFERENCE#     CREDIT          DATE    DEBIT  \
0  1,763.20               NaN  +1,763.20  01 Dec, 2024      NaN   
1  1,358.20  UPI-433665205422        NaN  01 Dec, 2024  -405.00   
2  1,464.20  UPI-433669845433    +106.00  01 Dec, 2024      NaN   
3  1,548.20  UPI-433669943628     +84.00  01 Dec, 2024      NaN   
4  1,575.20  UPI-433670096561     +27.00  01 Dec, 2024      NaN   

                                 TRANSACTION_DETAILS  
0                                OPENING BALANCE ...  
1  UPI/RAMESHWARAM ENT/433658302817/Payment for\r674  
2                    UPI/KRISHKUMAR/433649942953/UPI  
3       

In [18]:
import pandas as pd

df = pd.read_csv("extracted_tables_combined.csv")
# Display the first few rows of the DataFrame   
df

,BALANCE,CHEQUE/REFERENCE#,CREDIT,DATE,DEBIT,TRANSACTION_DETAILS
0,"1,763.20",NaN,"+1,763.20","01 Dec, 2024",NaN,OPENING BALANCE ...
1,"1,358.20",UPI-433665205422,NaN,"01 Dec, 2024",-405.00,UPI/RAMESHWARAM ENT/433658302817/Payment for\r674
2,"1,464.20",UPI-433669845433,+106.00,"01 Dec, 2024",NaN,UPI/KRISHKUMAR/433649942953/UPI
3,"1,548.20",UPI-433669943628,+84.00,"01 Dec, 2024",NaN,UPI/KRISHKUMAR/433650069544/UPI
4,"1,575.20",UPI-433670096561,+27.00,"01 Dec, 2024",NaN,UPI/SHAURYAPRATAPSI/433665658570/UPI
...,...,...,...,...,...,...
113,"2,485.47",UPI-436525427708,NaN,"30 Dec, 2024",-70.0,UPI/Mr. SHONN SUKHE/436561214143/UPI
114,"2,245.47",UPI-436652806258,NaN,"31 Dec, 2024",-240.0,UPI/CAFE COFFEE HUT/436696902597/UPI
115,"2,405.47",UPI-436652867072,160.0,"31 Dec, 2024",NaN,UPI/SOHAMSAMIRKADAM/436669906609/UPI
116,"2,365.47",UPI-436675770872,NaN,"31 Dec, 2024",-40.0,UPI/MOHINI SANDEEP /436690262241/UPI


## Rule based categorization by "Transaction Details" column

In [ ]:
import pandas as pd
from io import StringIO

# Define keyword-to-category mapping
category_keywords = {
    'Food': ['CAFE', 'RESTAURANT', 'JUICE', 'TEA', 'COFFEE', 'MC DONALDS', 'PUNERI MITHAI', 'SUPER'],
    'Investments': ['GROWW', 'MUTUALFUND', 'NACH-MUT-DR'],
    'Entertainment': ['TRUEVIBEZ'],
    'Health': ['BEYOND HEALTH'],
    'Income': ['NEFT', 'UPI_CRADJ', 'Int.Pd'],
    'Miscellaneous': []  # Default category for unmatched transactions
}

def categorize_transaction(details):
    if not isinstance(details, str) or not details:
        return 'Miscellaneous'
    
    details = details.upper()  # Convert to uppercase for case-insensitive matching
    for category, keywords in category_keywords.items():
        for keyword in keywords:
            if keyword in details:
                return category
    return 'Miscellaneous'


# Read CSV data
df = pd.read_csv("extracted_tables_combined.csv")

# Clean data: Remove any leading/trailing whitespace and handle missing values
df['TRANSACTION_DETAILS'] = df['TRANSACTION_DETAILS'].fillna('').str.strip()

# Apply categorization
df['CATEGORY'] = df['TRANSACTION_DETAILS'].apply(categorize_transaction)

# Save or display the result
# df.to_csv('categorized_transactions.csv', index=False)
print(df[['DATE', 'TRANSACTION_DETAILS', 'CREDIT', 'DEBIT', 'CATEGORY']])

# Optional: Summarize by category
summary = df.groupby('CATEGORY').agg({
    'CREDIT': 'sum',
    'DEBIT': 'sum',
    'TRANSACTION_DETAILS': 'count'
}).rename(columns={'TRANSACTION_DETAILS': 'Transaction Count'})
print("\nSummary by Category:")
print(summary)

             DATE                                TRANSACTION_DETAILS  \
0    01 Dec, 2024                                OPENING BALANCE ...   
1    01 Dec, 2024  UPI/RAMESHWARAM ENT/433658302817/Payment for\r674   
2    01 Dec, 2024                    UPI/KRISHKUMAR/433649942953/UPI   
3    01 Dec, 2024                    UPI/KRISHKUMAR/433650069544/UPI   
4    01 Dec, 2024               UPI/SHAURYAPRATAPSI/433665658570/UPI   
..            ...                                                ...   
113  30 Dec, 2024               UPI/Mr. SHONN SUKHE/436561214143/UPI   
114  31 Dec, 2024               UPI/CAFE COFFEE HUT/436696902597/UPI   
115  31 Dec, 2024               UPI/SOHAMSAMIRKADAM/436669906609/UPI   
116  31 Dec, 2024               UPI/MOHINI SANDEEP /436690262241/UPI   
117  31 Dec, 2024         Int.Pd:6845525302:01-10-2024 to 31-12-2024   

        CREDIT    DEBIT       CATEGORY  
0    +1,763.20      NaN  Miscellaneous  
1          NaN  -405.00  Miscellaneous  
2      +106.